<a href="https://colab.research.google.com/github/kpaulfran/sapceship_titanic/blob/main/sapceship_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/kpaulfran/sapceship_titanic/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/kpaulfran/sapceship_titanic/main/test.csv')

In [4]:
train.shape

(8693, 14)

1) Fuente de datos

Kaggle: https://www.kaggle.com/competitions/spaceship-titanic/data?select=train.csv

2) Breve descripción de datos

set of personal records recovered from the ship's damaged computer system.

3) ¿Cuál es el objetivo?

Predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. 

4) ¿Este es un problema de clasificación o regresión?

classifier

5) ¿Cuántas características hay?

14

6) ¿Cuántas filas de datos hay?

8693

7) ¿Qué retos, si es que hay alguno, prevén en la limpieza, exploración o modelización con este conjunto de datos?

Habra que realizar imputaciones en ciertas variables ya que superan el 1% de datos missing

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [9]:
#Exploración de datos por cada columna

train['PassengerId'].unique() # Sera separa al final #Grupo/# de persona en el grupo para evaluar si influye al graficarlo

array(['0001_01', '0002_01', '0003_01', ..., '9279_01', '9280_01',
       '9280_02'], dtype=object)

In [17]:
train['HomePlanet'].value_counts() #8492 datos / 201 datos na

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

In [20]:
round(1-(train['HomePlanet'].value_counts().sum()/8693),2) #Los datos faltantes son menor al 1% por lo que estas filas seran eliminadas para este caso

0.02

In [23]:
train.dropna(subset=['HomePlanet'], inplace=True)
train.shape

(8492, 14)

In [27]:
train['CryoSleep'].value_counts().sum()/8492 #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'most_frequent', se elige imputar por que ya hemos eliminado parte de nuestros datos

0.9746820536975978

In [29]:
train['Cabin'].unique() #Será separada al final, para luego graficarlo

array(['B/0/P', 'F/0/S', 'A/0/S', ..., 'G/1499/S', 'G/1500/S', 'E/608/S'],
      dtype=object)

In [30]:
train['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [32]:
train['Destination'].value_counts()

TRAPPIST-1e      5765
55 Cancri e      1769
PSO J318.5-22     780
Name: Destination, dtype: int64

In [33]:
train['Destination'].isna().sum()/train['Destination'].value_counts().sum() #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'most_frequent', se elige imputar por que ya hemos eliminado parte de nuestros datos

0.02140967043541015

In [34]:
train['Age'].describe()

count    8315.000000
mean       28.828743
std        14.490024
min         0.000000
25%        19.000000
50%        27.000000
75%        38.000000
max        79.000000
Name: Age, dtype: float64

In [35]:
train['Age'].isna().sum()  #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'mean'

177

In [38]:
train['VIP'].isna().sum()  #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'most_frequent', se elige imputar por que ya hemos eliminado parte de nuestros datos

200

In [37]:
train['VIP'].value_counts()

False    8098
True      194
Name: VIP, dtype: int64

In [ ]:
#Columnas de gastos en el spaceship seran sumadas 'total_billed'

In [39]:
#Eliminaremos Name para este análisis

train.drop(columns=['Name'], inplace = True)

In [40]:
train['Transported'].isna().sum() #Datos completos en columna objetivo

0

In [41]:
train['Transported'].value_counts() #Datos balanceados para una correcta clasificacion

True     4275
False    4217
Name: Transported, dtype: int64

In [55]:
#Segunda fase de la exploración y limpieza:

#Separar datos de las columnas 'PassengerID' y 'Cabin'

train['GroupId'] = train['PassengerId'].str.slice(stop = 4)
train['GroupId']

0       0001
1       0002
2       0003
3       0003
4       0004
        ... 
8688    9276
8689    9278
8690    9279
8691    9280
8692    9280
Name: GroupId, Length: 8492, dtype: object

In [57]:
train['PersonNumber'] = train['PassengerId'].str.slice(start = 5)
train['PersonNumber']

0       01
1       01
2       01
3       02
4       01
        ..
8688    01
8689    01
8690    01
8691    01
8692    02
Name: PersonNumber, Length: 8492, dtype: object